In [31]:
## GENERAL
import pandas as pd 
import numpy as np
import pickle


## NLP
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer 
from nltk.stem import PorterStemmer


## Machine Learning
from sklearn.model_selection import train_test_split
from sklearn import svm
import joblib
from sklearn.metrics import accuracy_score


# Text to TF IDF
data = pd.read_csv("/Users/nomena/Projects/App-ML-mail/App-ML-mail/mail-data/mail-all.csv")

class CountVectorizer:
    def __init__(self):
        self.tokenizer = TweetTokenizer()
        self.stop_words = set(stopwords.words('french'))
        self.vocab = None
        self.ps = PorterStemmer()

    def preprocess_text(self, text):

        text = text.lower()
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        tokens = self.tokenizer.tokenize(text)
        
        tokens = [self.ps.stem(token) for token in tokens if token.isalpha() and token not in self.stop_words]
        
        return tokens
    
    def fit_transform(self, documents, most_important):
        tokenized_documents = [self.preprocess_text(doc) for doc in documents]

        # Filter tokens based on most important words
        if most_important is not None:
            tokenized_documents = [[token for token in doc if token in most_important] for doc in tokenized_documents]

        self.vocab = Counter()
        for doc in tokenized_documents:
            self.vocab.update(doc)

        # Transformation des documents en bag-of-words
        count_vectors = []
        for doc in tokenized_documents:
            count_vector = [doc.count(term) for term in self.vocab]
            count_vectors.append(count_vector)

        return count_vectors


    def transform(self, documents, most_important):
        tokenized_documents = [self.preprocess_text(doc) for doc in documents]

        # Filter tokens based on most important words
        if most_important is not None:
            tokenized_documents = [[token for token in doc if token in most_important] for doc in tokenized_documents]

        # Transformation des documents en bag-of-words
        count_vectors = []
        for doc in tokenized_documents:
            count_vector = [doc.count(term) for term in self.vocab]
            count_vectors.append(count_vector)

        return count_vectors



    def get_feature_names_out(self):
        return list(self.vocab.keys())
    

class TfidfVectorizer:
    def __init__(self, count_vectorizer):
        self.count_vectorizer = count_vectorizer
        self.vocab_size = None
        self.idf = None
        self.feature_names = None

    def fit_transform(self, documents, most_important):
        # Step 1: Calcul de la matrice term frequency
        count_vectors = self.count_vectorizer.fit_transform(documents, most_important)

        self.feature_names = self.count_vectorizer.get_feature_names_out()

        # Step 2: Calcul de l' inverse document frequency
        self.vocab_size = len(self.count_vectorizer.vocab)
        doc_freq = np.sum(np.array(count_vectors) > 0, axis=0)
        self.idf = np.log((len(documents) + 1) / (doc_freq + 1)) + 1

        # Step 3: Normalisation de la matrice term frequency avec l'inverse document frequency
        tfidf_matrix = np.array(count_vectors) * self.idf

        # Step 4: Normalisation de la matrice TF-IDF par la norme Euclidiènne
        norms = np.linalg.norm(tfidf_matrix, axis=1, keepdims=True)
        norms[norms == 0] = 1e-12  # Remplacement des zéros
        tfidf_matrix /= norms

        return tfidf_matrix

    def transform(self, documents):
        # Step 1: Calcul de la matrice term frequency
        count_vectors = self.count_vectorizer.transform(documents)

        # Step 2: Normalisation de la matrice term frequency avec l'inverse document frequency
        tfidf_matrix = np.array(count_vectors) * self.idf

        # Step 3: Normalisation de la matrice TF-IDF par la norme Euclidiènne
        norms = np.linalg.norm(tfidf_matrix, axis=1, keepdims=True)
        norms[norms == 0] = 1e-12  # Replace zero norms with a small positive value
        tfidf_matrix /= norms

        return tfidf_matrix



text = data['texte']

count_vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(count_vectorizer)

tfidf_vectors = vectorizer.fit_transform(text, None)
feature_names = vectorizer.feature_names

tfidf_dataframe = pd.DataFrame(tfidf_vectors, columns=feature_names)
tfidf_dataframe.insert(0, 'classe_of_mail', data['classe_mail'])

print("\nNb de lignes : "+str(len(tfidf_dataframe.axes[0])))

# MAJ Modèle
total_data = tfidf_dataframe

X = total_data.iloc[:, 1:].values
y = total_data.iloc[:, 0].values

print("\nAvant feature tuning")
print('Nb mots : '+str(len(tfidf_dataframe.columns)-1))
print(tfidf_dataframe.shape)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

clf_svc = svm.SVC(kernel='linear')
clf_svc = clf_svc.fit(x_train, y_train)

y_pred = clf_svc.predict(x_test)
print("Précision modèle : "+str(accuracy_score(y_test, y_pred)))



#################################

feature_importances_svc = clf_svc.coef_
feature_names = total_data.columns[1:]
df_feature_importances_svc = pd.DataFrame(feature_importances_svc[0], index=feature_names)
df_feature_importances_svc_sorted = df_feature_importances_svc.reindex(df_feature_importances_svc[0].abs().sort_values(ascending=False).index)
most_important_features_svc = df_feature_importances_svc_sorted[:500]
df_most_important_features_svc = most_important_features_svc.T

count_vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(count_vectorizer)

tfidf_vectors = vectorizer.fit_transform(text, list(df_most_important_features_svc.columns))
feature_names = vectorizer.feature_names

tfidf_dataframe = pd.DataFrame(tfidf_vectors, columns=feature_names)
tfidf_dataframe.insert(0, 'classe_of_mail', data['classe_mail'])


# MAJ Modèle - IMPORTANT
total_data = tfidf_dataframe

X = total_data.iloc[:, 1:].values
y = total_data.iloc[:, 0].values

print("\nAprès feature tuning")
print('Nb mots : '+str(len(tfidf_dataframe.columns)-1))
print(tfidf_dataframe.shape)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

clf_svc = svm.SVC(kernel='linear')
clf_svc = clf_svc.fit(x_train, y_train)

y_pred = clf_svc.predict(x_test)
print("Précision modèle  : "+str(accuracy_score(y_test, y_pred))+"\n")

filename = '/Users/nomena/Projects/App-ML-mail/App-ML-mail/Data/tf-idf_vectorizer.dat'

# Objet à sérialiser
x = vectorizer

# sérialiser
with open(filename, 'wb') as f:
    pickle.dump(x, f)
 
tfidf_dataframe.to_csv('/Users/nomena/Projects/App-ML-mail/App-ML-mail/mail-data/mail-tfidf.csv')
joblib.dump(clf_svc, "/Users/nomena/Projects/App-ML-mail/App-ML-mail/Data/ai_mail_svc.joblib")


Nb de lignes : 395

Avant feature tuning
Nb mots : 1214
(395, 1215)
Précision modèle : 0.9873417721518988

Après feature tuning
Nb mots : 500
(395, 501)
Précision modèle  : 0.9873417721518988



['/Users/nomena/Projects/App-ML-mail/App-ML-mail/Data/ai_mail_svc.joblib']